In [47]:
import os, glob
from pathlib import Path
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Alignment, Font
import numpy as np

# Ruta base (ajústala si cambia)
BASE_DIR = Path(r"C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson")

# Lista de 31 encabezados, en el orden en que los quieres (eNBId queda en W si mantienes este orden)
HEADERS = [
    "eNodeB Name", "CellName", "activePlmnList_mcc", "additionalPlmnList_mcc",
    "administrativeState", "cellBarred", "cellId", "cellSubscriptionCapacity",
    "channelSelectionSetSize", "dlChannelBandwidth", "earfcndl", "earfcnul",
    "freqBand", "noOfPucchCqiUsers", "noOfPucchSrUsers", "operationalState",
    "physicalLayerCellIdGroup", "physicalLayerSubCellId", "sectorCarrierRef",
    "tac", "timeOfLastModification", "ulChannelBandwidth",
    "eNBId",  # <= posición deseada para W
    "eNodeB Name Unique", "LAT", "LON", "PCI", "AT&T_Site_Name",
    "MOCN Activo por Celda", "Al menos una celda de MOCN encendida", "MME TEF"
]




In [41]:

def appendfiles(filenamepattern: str) -> str:
    """
    Integra todos los TXT que matchean pattern + '_*.txt' en un solo archivo.
    Devuelve el nombre del archivo integrado (sin ruta).
    """
    searchpattern = str(BASE_DIR / f"{filenamepattern}_*.txt")
    filestoread = glob.glob(searchpattern)

    outputfile_name = f"Integrated_{filenamepattern}_files.txt"
    output_path = BASE_DIR / outputfile_name

    print("Buscando:", searchpattern)
    print("Archivos:", filestoread)

    with open(output_path, "w", encoding="utf-8") as outputfile:
        for name in filestoread:
            with open(name, "r", encoding="utf-8") as f:
                outputfile.write(f.read())
                print("Agregado:", name)

    print("Integrado =>", outputfile_name)
    return outputfile_name


def cleanfile(filename: str, ignorelines=None) -> str:
    """
    Elimina líneas que contengan cualquiera de los patrones indicados.
    Devuelve el nombre del archivo limpio (sin ruta).
    """
    if ignorelines is None:
        ignorelines = ["SubNetwork,", "instance(s)", "NodeId"]

    inputfile = BASE_DIR / filename
    cleanfile_name = f"Clean_{filename}"
    cleanfile_path = BASE_DIR / cleanfile_name

    with open(inputfile, 'r', encoding="utf-8") as f_in:
        lines = f_in.readlines()

    kept = []
    for line in lines:
        if any(p in line for p in ignorelines):
            continue
        kept.append(line)

    with open(cleanfile_path, 'w', encoding="utf-8") as f_out:
        f_out.writelines(kept)

    print(f"Limpieza OK -> {cleanfile_name} ({len(kept)} líneas)")
    return cleanfile_name


def convert_to_excel(cleanfile_name: str) -> str:
    """
    Lee TXT tab-delimited sin encabezados y guarda a Excel.
    Devuelve el nombre del archivo Excel (sin ruta).
    """
    cleanfile_path = BASE_DIR / cleanfile_name
    out_xlsx = f"Converted_{cleanfile_name}.xlsx"
    out_path = BASE_DIR / out_xlsx

    df = pd.read_csv(cleanfile_path, delimiter='\t', header=None)
    df.to_excel(out_path, index=False, header=None)
    print(f"Convertido a Excel -> {out_xlsx}  (shape={df.shape})")
    return out_xlsx



In [42]:

# EUtranCellFDD
eu_txt = appendfiles('EUtranCellFDD')
eu_clean = cleanfile(eu_txt)
eu_xlsx = convert_to_excel(eu_clean)

# ENodeBFunction
nb_txt = appendfiles('ENodeBFunction')
nb_clean = cleanfile(nb_txt)
nb_xlsx = convert_to_excel(nb_clean)

# nodeid (si lo sigues usando más adelante)
nd_txt = appendfiles('nodeid')
nd_clean = cleanfile(nd_txt)
nd_xlsx = convert_to_excel(nd_clean)



Buscando: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\EUtranCellFDD_*.txt
Archivos: ['C:\\Users\\SCaracoza\\Documents\\AT&T\\LST Cell Ran\\Ericsson\\EUtranCellFDD_14.txt', 'C:\\Users\\SCaracoza\\Documents\\AT&T\\LST Cell Ran\\Ericsson\\EUtranCellFDD_9.txt']
Agregado: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\EUtranCellFDD_14.txt
Agregado: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\EUtranCellFDD_9.txt
Integrado => Integrated_EUtranCellFDD_files.txt
Limpieza OK -> Clean_Integrated_EUtranCellFDD_files.txt (51830 líneas)


C:\Users\SCaracoza\AppData\Local\Temp\ipykernel_26912\2274692285.py:62: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(cleanfile_path, delimiter='\t', header=None)


Convertido a Excel -> Converted_Clean_Integrated_EUtranCellFDD_files.txt.xlsx  (shape=(51814, 23))
Buscando: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\ENodeBFunction_*.txt
Archivos: ['C:\\Users\\SCaracoza\\Documents\\AT&T\\LST Cell Ran\\Ericsson\\ENodeBFunction_14.txt', 'C:\\Users\\SCaracoza\\Documents\\AT&T\\LST Cell Ran\\Ericsson\\ENodeBFunction_9.txt']
Agregado: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\ENodeBFunction_14.txt
Agregado: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\ENodeBFunction_9.txt
Integrado => Integrated_ENodeBFunction_files.txt
Limpieza OK -> Clean_Integrated_ENodeBFunction_files.txt (6953 líneas)
Convertido a Excel -> Converted_Clean_Integrated_ENodeBFunction_files.txt.xlsx  (shape=(6937, 3))
Buscando: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\nodeid_*.txt
Archivos: ['C:\\Users\\SCaracoza\\Documents\\AT&T\\LST Cell Ran\\Ericsson\\nodeid_14.txt', 'C:\\Users\\SCaracoza\\Documents\\AT&T\\LST Cell Ran\\Ericsson\\n

In [43]:

# Archivo base desde la conversión de EUtranCellFDD
wb = load_workbook(BASE_DIR / eu_xlsx)  # ej. Converted_Clean_Integrated_EUtranCellFDD_files.txt.xlsx
ws = wb.active

ultima_fila = ws.max_row

# Mover B -> +30 columnas (B1:B{ultima_fila} => AF1:AF{ultima_fila})
rango = f"B1:B{ultima_fila}"
ws.move_range(rango, rows=0, cols=30)

# Mover C:AF -> -1 columna (C..AF => B..AE)
rango = f"C1:AF{ultima_fila}"
ws.move_range(rango, rows=0, cols=-1)

wb.save(BASE_DIR / "Modified_workfile.xlsx")
print("Reacomodo OK -> Modified_workfile.xlsx")



Reacomodo OK -> Modified_workfile.xlsx


In [49]:


# Leemos el archivo reacomodado SIN headers
df_base = pd.read_excel(BASE_DIR / "Modified_workfile.xlsx", header=None)

# Verificación de columnas
n_cols = df_base.shape[1]
print("Columnas detectadas en Modified_workfile.xlsx:", n_cols)

# Si tu tabla reacomodada debe tener exactamente len(HEADERS) columnas:
expected = len(HEADERS)
if n_cols < expected:
    # si faltan columnas, agregamos columnas vacías para completar
    for i in range(expected - n_cols):
        df_base[f"__tmp_empty_{i}"] = pd.NA
elif n_cols > expected:
    # si sobran columnas, las recortamos (ajusta si necesitas conservar alguna)
    df_base = df_base.iloc[:, :expected]

# Asignar los nombres canónicos
df_base.columns = HEADERS

# (Opcional) guardado de control sin formato
df_base.to_excel(BASE_DIR / "Modified_with_headers.xlsx", index=False)
print("Headers asignados en pandas -> Modified_with_headers_pandas.xlsx")



Columnas detectadas en Modified_workfile.xlsx: 31
Headers asignados en pandas -> Modified_with_headers_pandas.xlsx


In [60]:




# --- eNBId desde ENodeBFunction ---
df_nodeb = pd.read_excel(BASE_DIR / nb_xlsx, header=None, usecols=[0, 1, 2])
# Lee el archivo Excel convertido de ENodeBFunction (ruta 'nb_xlsx'),
# sin encabezado (header=None), y solo las 3 primeras columnas (0,1,2).

df_nodeb.columns = ["NodeId", "ENodeBFunctionId", "eNBIdnew"]
# Asigna nombres a las 3 columnas: NodeId (clave), ENodeBFunctionId (solo informativo),
# y eNBIdnew (el valor que queremos traer por JOIN).

df_base["eNodeB Name"] = df_base["eNodeB Name"].astype(str)
# Asegura que la columna clave en df_base sea string (evita mismatches de tipo).

df_nodeb["NodeId"] = df_nodeb["NodeId"].astype(str)
# Asegura que la clave en el catálogo (NodeId) también sea string.

df_nodeb = df_nodeb.drop_duplicates(subset=["NodeId"], keep="first")
# Si hay filas duplicadas por NodeId en el catálogo, conserva la primera
# para evitar que el merge genere duplicados.

df_tmp = df_base.merge(df_nodeb[["NodeId", "eNBIdnew"]],
                       left_on="eNodeB Name", right_on="NodeId", how="left")
# LEFT JOIN: por cada fila de df_base, busca en df_nodeb la fila con el mismo NodeId.
# - Clave izquierda: eNodeB Name (df_base)
# - Clave derecha: NodeId (df_nodeb)
# - how="left": conserva todas las filas de df_base aunque no haya match.

df_base["eNBId"] = df_tmp["eNBIdnew"]
# Copia (asigna) a df_base la columna eNBId con el valor traído (eNBIdnew).
# Nota: si no hubo match, quedará NaN.

# --- eNodeB Name Unique (solo cuando cambia) ---
_name = df_base["eNodeB Name"].astype(str).fillna("").str.strip()
# Toma la columna eNodeB Name, la convierte a str, reemplaza NaN por "",
# y recorta espacios en extremos para comparar bien.

is_new = _name.ne(_name.shift())
# Crea una serie booleana True/False que vale True cuando
# el nombre actual es diferente al de la fila anterior (inicio de bloque).

df_base["eNodeB Name Unique"] = np.where(is_new & _name.ne(""), df_base["eNodeB Name"], "")
# Si cambia el nombre (is_new=True) y no está vacío: escribe el nombre.
# En caso contrario: deja cadena vacía "" (equivalente a tu SI(A2=A1,"",A2)).

# --- LAT/LON/AT&T_Site_Name desde All_Ericsson_4G_{YYYYMM} (mes anterior) ---
from datetime import date
import re

# Importa utilidades: 'date' para calcular mes anterior y 're' para regex en búsqueda de archivos.

today = date.today()
# Fecha de hoy (del sistema donde corre el notebook).

prev_year, prev_month = (today.year, today.month - 1) if today.month > 1 else (today.year - 1, 12)
# Calcula el mes anterior: si hoy es enero, retrocede al diciembre del año previo.

yyyymm = f"{prev_year}{prev_month:02d}"
# Formatea YYYYMM (p.ej. 202508 para agosto 2025).

patterns = [f"All_Ericsson_4G_{yyyymm}*.xlsx", f"All_Ericsson_4G_{yyyymm}*.xls", f"All_Ericsson_4G_{yyyymm}*"]
# Define patrones de búsqueda para localizar el archivo del mes anterior (por si tiene sufijo o variaciones).

cands = []
for pat in patterns:
    cands += list(BASE_DIR.glob(pat))
# Busca en BASE_DIR archivos que cumplan los patrones anteriores y acumúlalos en cands.

if not cands:
    all_cands = [p for p in BASE_DIR.glob("All_Ericsson_4G_*")
                 if (p.suffix.lower() in {".xlsx", ".xls", ".csv", ".xlsm", ".xlsb"}) or re.search(
            r"All_Ericsson_4G_\d{6}", p.name)]


    # Si no hay match exacto del mes anterior:
    # Toma cualquier archivo que parezca All_Ericsson_4G_* y tenga extensión válida
    # o lleve YYYYMM en el nombre.

    def ext_ym(p):
        m = re.search(r"All_Ericsson_4G_(\d{6})", p.name);
        return int(m.group(1)) if m else -1


    # Función auxiliar para extraer YYYYMM como entero (o -1 si no lo encuentra).

    ae_path = sorted(all_cands, key=ext_ym)[-1] if all_cands else None
    # Ordena por YYYYMM y toma el más reciente disponible (fallback).
else:
    ae_path = cands[0]
# Si hubo candidatos exactos del mes anterior, toma el primero.

if ae_path is None:
    raise FileNotFoundError(f"No se encontró All_Ericsson_4G para {yyyymm} en {BASE_DIR}")
# Si de plano no hay ningún archivo All_Ericsson_4G válido, detén el proceso con error claro.

print("Usando All_Ericsson:", ae_path.name)
# Log informativo: qué archivo fue seleccionado.

usecols = ["eNodeB Name", "LAT", "LON", "AT&T_Site_Name"]
# Columnas que nos interesan traer del archivo All_Ericsson.

if ae_path.suffix.lower() == ".csv":
    ae_df = pd.read_csv(ae_path, usecols=usecols)
else:
    ae_df = pd.read_excel(ae_path, usecols=usecols)
# Lee el archivo All_Ericsson según su extensión (CSV o Excel) trayendo solo las columnas necesarias.

ae_df["eNodeB Name"] = ae_df["eNodeB Name"].astype(str).str.strip()
# Normaliza eNodeB Name en el catálogo All_Ericsson (str + strip).

ae_df = ae_df.drop_duplicates(subset=["eNodeB Name"], keep="first")
# Si hubiese duplicados por eNodeB Name en All_Ericsson, quédate con el primero.

df_base["eNodeB Name"] = df_base["eNodeB Name"].astype(str).str.strip()
# Normaliza eNodeB Name en tu base (asegura match correcto).

merged = df_base.merge(ae_df, on="eNodeB Name", how="left", suffixes=("", "_ae"))
# LEFT JOIN entre tu base y el All_Ericsson por eNodeB Name.
# Trae columnas LAT, LON y AT&T_Site_Name con sufijo "_ae" si colisionan.

for col in ["LAT", "LON", "AT&T_Site_Name"]:
    if col + "_ae" in merged.columns:
        merged[col] = merged[col].fillna(merged[col + "_ae"])
        merged.drop(columns=[col + "_ae"], inplace=True)
# Para cada columna objetivo:
# - Si ya existía en df_base, solo rellena los NaN con el valor del catálogo (_ae).
# - Luego elimina la columna auxiliar *_ae.
# (Si quisieras sobrescribir siempre, harías: merged[col] = merged[col + "_ae"]).

df_base = merged
# Actualiza df_base con el DataFrame enriquecido.

# --- Guardar preliminar (lo tomará la [7] para formateo final) ---
final_path = BASE_DIR / "Datos_Modified.xlsx"
# Define la ruta del Excel preliminar (sin estilos).

df_base.to_excel(final_path, index=False)
# Escribe el Excel con todas las columnas (aquí todavía sin formato openpyxl).

print("Guardado enriquecido ->", final_path, "shape=", df_base.shape)
# Log: confirma guardado y muestra dimensiones finales.





Usando All_Ericsson: All_Ericsson_4G_202508.xlsx
Guardado enriquecido -> C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\Datos_Modified.xlsx shape= (51814, 31)


In [57]:


final_excel = BASE_DIR / "Datos_Modified.xlsx"
tmp_excel = BASE_DIR / "~tmp_Datos_Modified.xlsx"

# Releer, forzar columnas y orden
df_out = pd.read_excel(final_excel)

# Garantiza que TODAS las columnas existan
for col in HEADERS:
    if col not in df_out.columns:
        df_out[col] = pd.NA

# Reordena exactamente como HEADERS
df_out = df_out[HEADERS]

# Escribe temporal
df_out.to_excel(tmp_excel, index=False)

# Reaplicar estilo vertical de headers
wb = load_workbook(tmp_excel)
ws = wb.active

# Congelar encabezado
ws.freeze_panes = "A2"

# Aplicar estilo a fila 1
for col_idx, header in enumerate(HEADERS, start=1):
    cell = ws.cell(row=1, column=col_idx)
    cell.value = header
    cell.font = Font(name="Aptos Narrow", size=11)  # bold=True si quieres negrita
    cell.alignment = Alignment(textRotation=90, horizontal="center", vertical="bottom", wrap_text=True)

# (Opcional) Ajustar ancho mínimo por header vertical
# for col in ws.iter_cols(min_row=1, max_row=1):
#     ws.column_dimensions[col[0].column_letter].width = 6

wb.save(final_excel)

# Limpia temporal
try:
    tmp_excel.unlink()
except Exception as e:
    print("No se pudo borrar temporal:", e)

print("Ajuste final OK -> Headers verticales y columnas forzadas/ordenadas.")



Ajuste final OK -> Headers verticales y columnas forzadas/ordenadas.
